# Translation Memory Retrieval using Weighted N-Grams

In [188]:
import nltk
import math
from collections import Counter
import string
import numpy as np

In [189]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/ashes/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [190]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

stop_words = stopwords.words('english')

### Getting the M_ngrams and C_ngrams

In [191]:
def get_M_ngrams(sentence):
    ngrams_list_sent = []
    M_ngrams = []
    counter_ngrams = []
    
    ngrams = list(nltk.ngrams(sentence.split(), 4))
    ngrams_list_sent.append(list(ngrams))
    M_ngrams = [y for x in ngrams_list_sent for y in x]
    
    for ngrams in M_ngrams:
        counter_ngrams.append(Counter(ngrams))
        
    return M_ngrams



In [192]:
def get_C_ngrams(candidate_sentence):
    ngrams_list_sent = []
    C_ngrams = []
    counter_ngrams = []
    
    ngrams = list(nltk.ngrams(candidate_sentence.split(), 4))
    ngrams_list_sent.append(list(ngrams))
    C_ngrams = [y for x in ngrams_list_sent for y in x]
    ngrams_sents = []
    ngrams_list_sent = []
    
    for ngrams in C_ngrams:
        counter_ngrams.append(Counter(ngrams))
    
    return C_ngrams


In [193]:
input_line = input()

#convert input to lowercase
input_line = input_line.lower()

#tokenise
input_tokens = word_tokenize(input_line)

content_words = [word for word in input_tokens if word not in stop_words] #Removing Stopwords

new_M_sentence = ' '.join(content_words)
M_ngrams = get_M_ngrams(new_M_sentence)
w_M_ngrams = compute_w_sum(M_ngrams)

# print(w_M_ngrams)

I request you to remove the drive safely


## Weighted N-Gram Precision

### Load TM

In [194]:
src_tm_words = [] #Content Words in Source TM

with open('tm_data/tm_src_2000_pp.txt') as src_tm:
    line = src_tm.readline()
    
    while line:
        line = line.rstrip() #Removing Trailing Whitespace
        
        words = line.split('\t')
        src_tm_words.append(words)
        
        line = src_tm.readline()

### Get sentences and IDF values

In [195]:
with open("tm_data/tm_src_2000.txt") as source_file:
    sentences = source_file.read().splitlines()

In [196]:
idf_values = {}

tokenized_sentences = [word_tokenize(sentence) for sentence in sentences]
all_tokens_set = set([item for sublist in tokenized_sentences for item in sublist])
for tkn in all_tokens_set:
    contains_token = map(lambda doc: tkn in doc, tokenized_sentences)
    idf_values[tkn] = 1 + math.log(len(tokenized_sentences)/(sum(contains_token)))

print(idf_values)

{'unit': 8.600902459542082, 'Looked': 8.600902459542082, 'indeed': 6.654992310486769, 'baby': 8.600902459542082, 'BALCO': 8.600902459542082, 'hurry': 8.600902459542082, 'materials': 7.214608098422191, 'accused': 8.600902459542082, 'Right': 8.600902459542082, 'Text': 7.502290170873972, 'minister': 8.600902459542082, 'key': 8.600902459542082, 'picture': 7.907755278982137, 'Him': 6.809142990314028, 'decade': 8.600902459542082, 'radio': 7.907755278982137, 'educated': 8.600902459542082, 'Montreal': 8.600902459542082, 'mark': 8.600902459542082, 'version': 8.600902459542082, 'school': 7.502290170873972, 'somnolence': 8.600902459542082, 'scan': 8.600902459542082, 'worker': 8.600902459542082, 'Things': 8.600902459542082, 'Kalyanpur': 8.600902459542082, 'Indeed': 6.654992310486769, 'Child': 8.600902459542082, 'flowing': 7.907755278982137, 'knowing': 8.600902459542082, 'Prahlad': 8.600902459542082, 'paradox': 8.600902459542082, 'errors': 8.600902459542082, 'myur': 8.600902459542082, 'collection':

### To compute numerator and denominator

In [197]:
def ngrams_intersection(candidate_sentence):
    C_ngrams = get_C_ngrams(candidate_sentence)
    
    M_set = set(M_ngrams)
    C_set = set(C_ngrams)
    
    return list(M_set & C_set)

In [198]:
def compute_w_sum(ngrams_list):
    w = 0
    
    
    for ngram in ngrams_list:
        for token in ngram:
            if token in idf_values:
                w += idf_values[token]
#                 print(w)
    
    return w

### Final score for each sentence wrt to input sentence

In [199]:
def compute_wpn(candidate_sentence):
    C_ngrams = get_C_ngrams(candidate_sentence)
    intersection_ngrams = ngrams_intersection(candidate_sentence)
        
    Z = 0.75
    
    w_M_ngrams = compute_w_sum(M_ngrams)
    w_C_ngrams = compute_w_sum(C_ngrams)
    w_intersection_ngrams = compute_w_sum(intersection_ngrams)

    
    
    wpn = w_intersection_ngrams / ((Z*w_M_ngrams) + ((1-Z)*w_C_ngrams))
    
    return wpn

## Optimisation

In [200]:
N = 5 #Top N matches returned

wpn_all = []
indices_all = []

j = 0
count = 0

for i, candidate in enumerate(src_tm_words):
    
    #Check if Content Words present in Candidate
    for word in content_words:
        if(word in candidate):
            count += 1
            
            wpn = compute_wpn(sentences[i])
            
            wpn_all.append(wpn)
            indices_all.append(j)
            
            break
    
    j += 1
    
print('Running WNGP on ' + str(count) + ' Candidates out of a possible ' + str(j) + '!\n')


#Get top N results
wpn_all = np.array(wpn_all)

sorted_indices = np.argsort(wpn_all) #Sorts in ascending order and returns the indices of indices_all array
least_N_indices = sorted_indices[-N:] 

for i in least_N_indices:
    print([indices_all[i]], src_tm_words[indices_all[i]], wpn_all[i])

Running WNGP on 8 Candidates out of a possible 2000!

[1126] ['extension', 'failed', 'modify', 'network', 'request', 'modification', 'conflicted', 'another', 'extension', '.'] 0.0
[1179] ['rama', 'got', 'angry', 'approved', 'request'] 0.0
[1805] ['develop', 'recommendations', 'make', 'running', 'away', 'less', 'likely', 'ensure', 'runaways', "'", 'short', 'long', 'term', 'needs', 'safely', 'met', '.'] 0.0
[1807] ['“', 'safely', 'boarded', 'ship', 'say'] 0.0
[1916] ['remove', 'cough'] 0.0


### Retrieval of Target from TM

In [201]:
tgt_tm_array = []

with open('tm_data/tm_tgt_2000.txt') as tgt_tm:
    line = tgt_tm.readline()
    
    while line:
        tgt_tm_array.append(line)
        line = tgt_tm.readline()
        
for i in least_N_indices:
    print([indices_all[i]], tgt_tm_array[indices_all[i]])

[1126] यह एक्सटेंशन

[1179] विनती न मानने पर राम ने क्रोध किया और उनके क्रोध से भयभीत होकर समुद्र ने स्वयं आकर राम की विनती करने के पश्चात् नल और नील के द्वारा पुल बनाने का उपाय बताया ।

[1805] घर छोडकर भागने की संभावना घटाने और घर छोडकर भाग जाने वालों की अल्प तथा दीर्घ-कालिक ज़रुरतों को सुरक्षित तरीके से पूरा करने के लिए प्रस्ताव विकसित करना ।

[1807] फिर जब तू नौका पर सवार हो जाए और तेरे साथी भी तो कह

[1916] कब्ज को दूर करने के लिए पका हुआ बेल खाने से आराम मिलता है ।

